In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import regularizers
from matplotlib import pyplot as plt
import tensorflow as tf
from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tf.keras.losses import sparse_categorical_crossentropy
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D,BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
# from keras.optimizers import Adam, SGD
from tensorflow.keras.applications import ResNet50, MobileNet
from keras.models import Model

In [2]:

img_height,img_width=(128,128)
batch_size=12
train_data_dir="Dataset/"

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.3)
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='training')
test_generator = train_datagen.flow_from_directory(train_data_dir,
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation')

Found 14813 images belonging to 4 classes.
Found 6345 images belonging to 4 classes.


In [3]:
base_model=ResNet50(include_top=False,weights='imagenet')
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(4,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=predictions)

for layer in base_model.layers:
    layer.trainable=False

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_generator,
          epochs=25,
          batch_size=16,
          validation_data=test_generator)

model.save("models/resnet.h5")

Epoch 1/25
1235/1235 [==============================] - 1710s 1s/step - loss: 1.1133 - accuracy: 0.5436 - val_loss: 1.1167 - val_accuracy: 0.4884
Epoch 2/25
1235/1235 [==============================] - 1581s 1s/step - loss: 1.0058 - accuracy: 0.5894 - val_loss: 0.9863 - val_accuracy: 0.5872
Epoch 3/25
1235/1235 [==============================] - 1520s 1s/step - loss: 0.9481 - accuracy: 0.6045 - val_loss: 0.9038 - val_accuracy: 0.6296
Epoch 4/25
1235/1235 [==============================] - 1562s 1s/step - loss: 0.9214 - accuracy: 0.6157 - val_loss: 0.8915 - val_accuracy: 0.6397
Epoch 5/25
1235/1235 [==============================] - 1584s 1s/step - loss: 0.8915 - accuracy: 0.6301 - val_loss: 0.8922 - val_accuracy: 0.6287
Epoch 6/25
1235/1235 [==============================] - 1552s 1s/step - loss: 0.8828 - accuracy: 0.6301 - val_loss: 0.9230 - val_accuracy: 0.6142
Epoch 7/25
1235/1235 [==============================] - 1430s 1s/step - loss: 0.8643 - accuracy: 0.6371 - val_loss: 0.8626 -

KeyboardInterrupt: 

In [ ]:


plt.style.use("ggplot")
plt.figure()
plt.plot(history.history['accuracy'],'r',label='Testing accuracy',color='green')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("models/resnet_acc.png")
plt.show()


plt.style.use("ggplot")
plt.figure()
plt.plot(history.history['loss'],'r',label='Testing accuracy',color='green')
plt.plot(history.history['val_loss'],label='validation accuracy')
plt.xlabel('# Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("models/resnet_loss.png")
plt.show()


acc=history.history['accuracy'][-1]
print(acc)
